# Introduction
In this small task of recommendation, the dataset is obtained from [dataset for recommendation](https://grouplens.org/datasets/movielens/).

Recommendation is useful to help both provider and consumer to maximize provider's profits or user's knowledge/exploration, and this technique is very popular in big data processing with huge dataset.

In this small task, we will go through following steps based on the idea of matrix factorization [here](https://dl.acm.org/doi/10.1109/MC.2009.263) and collaborative filtering [here](https://ieeexplore.ieee.org/document/4781121).
- load the dataset and processing 
- exploration dataset to find some intuitations about the dataset
- train Alternative Least Square model for recommendation
- make recommendation for the most active user
- evaluation the trained model on test dataset based on metrics rmse, mae. (root mean square error and mean absolute error).
- some conclusions.  

# 1. Setup colab to work with pyspark

In [4]:
# setup for pyspark working on google colab
'''
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
'''
from google.colab import drive
drive.mount('/content/gdrive')

!cp '/content/gdrive/My Drive/pyspark/spark-2.4.5-bin-hadoop2.7.tgz' .

!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# setup java home and spark home directory in google collab
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# import some library pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark import SparkConf, SparkContext
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *

# create spark context set-up
conf = SparkConf().setAppName('sql_dataframe')
sc = SparkContext.getOrCreate(conf = conf)
sqlcontext = SQLContext(sc)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 2. Reading data ratings from local file

In [5]:
# reading file in local system
url = '/content/gdrive/My Drive/pyspark/recommendation/ratings.csv'

# take first header 
str_header = sc.textFile(url).first()

# read ratings and remove header
rdd_ratings = (sc.textFile(url).map(lambda x: x.split(','))
                               .filter(lambda x: x != str_header.split(','))                             
                               .map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
                               )

# show the first two lines of rdd ratings
print('First two lines of rdd_ratings: ', rdd_ratings.take(2))

# convert to rdd
df_ratings = rdd_ratings.toDF(['userID', 'movieID', 'rating'])
df_ratings.show(5)

First two lines of rdd_ratings:  [(1, 1, 4.0), (1, 3, 4.0)]
+------+-------+------+
|userID|movieID|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
+------+-------+------+
only showing top 5 rows



### **Alternative way to remove header with rdd**
- *create rdd header*, *create rdd with full content*, then, *full - header*
1. rdd_head = sc.parallelize([sc.textFile(url).first()])
2. rdd_alternative = sc.textFile(url)
3. rdd_alternative.subtract(rdd_header).take(2)

# 3. Exploration the data ratings, movies

In [10]:
## reading movies 
# reading file in local system
url = '/content/gdrive/My Drive/pyspark/recommendation/movies.csv'

# take first header 
str_header = sc.textFile(url).first()

# read ratings and remove header
rdd_movies = (sc.textFile(url).map(lambda x: x.split(','))
                               .filter(lambda x: x != str_header.split(','))                              
                               .map(lambda x: (int(x[0]), str(x[1]), str(x[2]))))
## convert rdd to dataframe
df_movies = rdd_movies.toDF(['movieID', 'title', 'genres'])
df_movies.show(3)

+-------+--------------------+--------------------+
|movieID|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



In [11]:
# assign df_ratings and df_movies to Tempview to use sql querries
df_ratings.registerTempTable('ratings')
df_movies.registerTempTable('movies')

# get lowest and highest ratings for each movie
# and the total number of users rated for that movie
df_summary = sqlcontext.sql(''' 
    SELECT  M.title,
            N.movieID,
            N.minrate,
            N.maxrate,
            N.numuser
    FROM  (SELECT ratings.movieID,
                  min(ratings.rating) AS minrate, 
                  max(ratings.rating) AS maxrate, 
                  count(DISTINCT ratings.userID) AS numuser 
           FROM ratings 
           GROUP BY ratings.movieID) N
    JOIN  movies M
    ON  N.movieID = M.movieID
    ORDER BY N.numuser DESC
                       
''')
df_summary.show(10)

+--------------------+-------+-------+-------+-------+
|               title|movieID|minrate|maxrate|numuser|
+--------------------+-------+-------+-------+-------+
| Forrest Gump (1994)|    356|    0.5|    5.0|    329|
|"Shawshank Redemp...|    318|    1.0|    5.0|    317|
| Pulp Fiction (1994)|    296|    0.5|    5.0|    307|
|"Silence of the L...|    593|    0.5|    5.0|    279|
|             "Matrix|   2571|    0.5|    5.0|    278|
|Star Wars: Episod...|    260|    0.5|    5.0|    251|
|Jurassic Park (1993)|    480|    0.5|    5.0|    238|
|   Braveheart (1995)|    110|    0.5|    5.0|    237|
|Terminator 2: Jud...|    589|    0.5|    5.0|    224|
|Schindler's List ...|    527|    0.5|    5.0|    220|
+--------------------+-------+-------+-------+-------+
only showing top 10 rows



In [12]:
# Show top of user who rated most frequent
df_active_user = sqlcontext.sql('''
    SELECT r.userID, 
           count(*) AS numrate
    FROM ratings r
    GROUP BY r.userID
    ORDER BY numrate 
    DESC  
''')

df_active_user.show(10)

+------+-------+
|userID|numrate|
+------+-------+
|   414|   2698|
|   599|   2478|
|   474|   2108|
|   448|   1864|
|   274|   1346|
|   610|   1302|
|    68|   1260|
|   380|   1218|
|   606|   1115|
|   288|   1055|
+------+-------+
only showing top 10 rows



In [13]:
# It is clearly that the userID = 414 has the highest
# number of rates  with 2698 times. Now, we can filter 
# to see how many times he rated greater than 4.5 stars
df_userID_414 =  sqlcontext.sql(''' 
    SELECT m.title,
           r.movieID,
           r.rating
    FROM ratings r
    JOIN movies m
    ON r.movieID = m.movieID
    WHERE (r.userID = 414 AND r.rating >= 4.5)
''')

df_userID_414.show()

+--------------------+-------+------+
|               title|movieID|rating|
+--------------------+-------+------+
|Up in the Air (2009)|  72011|   4.5|
| Blade Runner (1982)|    541|   5.0|
|Cyrano de Bergera...|   1277|   5.0|
|Léon: The Profess...|    293|   5.0|
|    Secretary (2002)|   5617|   5.0|
| Spider-Man 2 (2004)|   8636|   4.5|
|Wallace & Gromit:...|    720|   5.0|
|        "Dirty Dozen|   2944|   5.0|
|   Robin Hood (1973)|   3034|   5.0|
|Kubo and the Two ...| 162578|   4.5|
| Pulp Fiction (1994)|    296|   5.0|
|          Ran (1985)|   1217|   5.0|
|Sleepy Hollow (1999)|   3081|   5.0|
|Scent of a Woman ...|   3252|   5.0|
|Hope and Glory (1...|   4117|   5.0|
|Million Dollar Ba...|  30707|   4.5|
|To Sir with Love ...|   3296|   5.0|
|Midnight in the G...|   1711|   5.0|
| American Pie (1999)|   2706|   5.0|
|       WALL·E (2008)|  60069|   4.5|
+--------------------+-------+------+
only showing top 20 rows



# 4. train the model using ALS 

In [7]:
# import the model ALS to train
from pyspark.mllib.recommendation import ALS, Rating, MatrixFactorizationModel

# using Rating function to prepare input for ALS() model
rdd_ratings_Rating = rdd_ratings.map(lambda x: Rating(int(x[0]), int(x[1]), float(x[2])))
print('two first lines after applying Rating: ', rdd_ratings_Rating.take(2))

# split data into train_data and test_data
rdd_train, rdd_test = rdd_ratings_Rating.randomSplit([0.8, 0.2], seed = 12345)
print('number of train data: ', rdd_train.count())
print('number of test data: ', rdd_test.count())

# train ALS() 
number_of_latent_features = 20
number_of_iterations = 20 
model = ALS.train(rdd_ratings_Rating, number_of_latent_features, number_of_iterations)


two first lines after applying Rating:  [Rating(user=1, product=1, rating=4.0), Rating(user=1, product=3, rating=4.0)]
number of train data:  80828
number of test data:  20008


# 5. Make recommendations for active userID 

In [21]:
# use trained model  to return recommend
toprecommend = model.recommendProducts(414, 10)
df_recommend = sqlcontext.createDataFrame(toprecommend)
df_recommend.show()

# show the recommended movies
df_join = df_recommend.join(df_movies, df_recommend.product == df_movies.movieID, how='left')
df_join.drop('movieID', 'rating').show()

+----+-------+------------------+
|user|product|            rating|
+----+-------+------------------+
| 414|   3468|  5.78499878169281|
| 414|   8228| 5.565813532052579|
| 414|   1284| 5.506483477088946|
| 414|   3421| 5.468809189201311|
| 414|    954|5.4200399170853615|
| 414|   2997| 5.344885289603891|
| 414|    720|5.2965448293393065|
| 414|    912| 5.270203470393725|
| 414|   1249| 5.257831454528875|
| 414|   1090| 5.244817220249215|
+----+-------+------------------+

+----+-------+--------------------+--------------------+
|user|product|               title|              genres|
+----+-------+--------------------+--------------------+
| 414|    720|Wallace & Gromit:...|Adventure|Animati...|
| 414|   8228|     "Maltese Falcon| The (a.k.a. Dang...|
| 414|   3468|            "Hustler|         The (1961)"|
| 414|    912|   Casablanca (1942)|       Drama|Romance|
| 414|   1284|          "Big Sleep|         The (1946)"|
| 414|   3421| Animal House (1978)|              Comedy|
| 414|   1

# 6. Evaluation the predictions and the actual ratings

In [0]:
# prepare the test data
rdd_test_remove_prediction = rdd_test.map(lambda x: (int(x[0]), int(x[1])))
print('two first line of rdd_test after removing prediction: ', rdd_test_remove_prediction.take(2))

# prediction on redd_test_remove_prediction using model after fit 
rdd_prediction_test = model.predictAll(rdd_test_remove_prediction).map(lambda x: ((x[0], x[1]), x[2]))
print('two fisrt line of predictions using model: ', rdd_prediction_test.take(2))
print('number of predictions has valid label: ', rdd_prediction_test.count())

two first line of rdd_test after removing prediction:  [(1, 1), (1, 231)]
two fisrt line of predictions using model:  [((590, 1084), 4.252041544191991), ((32, 1084), 4.199594019243957)]
number of predictions has valid label:  20008


In [0]:
# evaluation
# join two rdds (rdd_test and rdd_prediction_test)
import math
rdd_actualvspredicted = rdd_test.map(lambda x: ((x[0], x[1]), x[2])).join(rdd_prediction_test)
print('first two lines of joined rdd: ', rdd_actualvspredicted.first())
print('number of lines after join: ', rdd_actualvspredicted.count()) 

# evaluate predictions using (rmse)
mse = (rdd_actualvspredicted.map(lambda x: (x[1][0] - x[1][1])**2)
                            .mean())
print('root mean square error: ', math.sqrt(mse))

# evaluate mean absolute error (mae)
mae = (rdd_actualvspredicted.map(lambda x: abs(x[1][0] - x[1][1]))
                            .mean())
print('mean absolute square error: ', mae) 

first two lines of joined rdd:  ((1, 1), (4.0, 4.67654777615138))
number of lines after join:  20008
root mean square error:  0.3409566405717262
mean absolute square error:  0.22442322712402069


# 7. Check whether any null data in rdd predictions  

In [0]:
# check null in the prediction
# first take all predictions in scale range [0-5]
rdd_subtract = rdd_prediction_test.filter(lambda x: (x[1] >= 0 and x[1] <= 5))
# subtract the scale range [0-5] to see the ouliers 
# whether it contains null/nan or not 
len_outlier = len(rdd_prediction_test.subtract(rdd_subtract).collect())
len_range_0_5 = rdd_subtract.count()
print('total test is: 20008?', len_outlier+len_range_0_5)


total test is: 20008? 20008


# 8. Conclusion
As we can see that the evaluation show that the root mean square error and the mean absolute error, both around 0.33, are quite small, and show the effectiveness of the method recommendation.